# TP 5 : Evaluation

In [ ]:
# only if using Google Colab:
!pip install myqlm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qat.lang import QRoutine, H, PH, CNOT, RX, RY, RZ, X, Y, Z, T, CCNOT, Program, S
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

def get_result(circuit, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=0, qubits=idx))
    states = []
    amplitudes = []
    for sample in result:
            states.append(sample.state.int)
            if idx:
                amplitudes.append(sample.probability)
            else:
                amplitudes.append(sample.amplitude)
    return np.array(states), np.array(amplitudes)

## Exercice 1 : Circuits aléatoires et distribution de Porter-Thomas

Dans cet exercice nous allons étudier un type de circuits dénommés circuits "Haar random", constitués de rotations aléatoirs et de portes intricatrices. Le but est démontrer que l'intrication donne naissance à un aléatoire quantique bien différent de l'aléatoire classique.

<center>

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/haar_random.png)

*Circuit aléatoire Haar random constitué de 5 layers. Chaque layer contient un mur de rotations aléatoires, et un mur de CNOTs*

</center>

**Question 1** : Implémenter la fonction ```haar_random_circuit``` qui renvoie une QRoutine correspondant à un circuit aléatoire tel que celui donné dans la figure ci-dessus. 
Chaque layer doit contenir:
- Une rotation aléatoire à 1 qubit (RX, RY, ou RZ) avec un angle alétoire $\theta \in [0, 2 \pi [$
- Un mur de CNOT qui intrique les qubits voisins selon un "brickwork pattern":
    - Layers pairs: CNOTs sur ($0 \rightarrow 1$, $2 \rightarrow 3$, $4 \rightarrow 5$..)
    - Layers impairs: CNOTs sur ($1 \rightarrow 2$, $3 \rightarrow 4$, $5 \rightarrow 6$..)

Je vous conseille d'utiliser les fonctions de np.random pour le coté aléatoire:
- np.random.random() génère un nombre aléatoire entre 0 et 1
- np.random.choice([a, b,c]) choisi un élément au hasard dans [a, b, c]


In [ ]:
def haar_random_circuit(nqbits, depth):
    rout = QRoutine()
    qbits = rout.new_wires(nqbits)

    # A compléter

    return rout

In [ ]:
# Vous pouvez vérifier ici que votre circuit à la bonne forme

circ = haar_random_circuit(nqbits=5, depth=5)
circ.display()

On peut maintenant extraire les probabilités de chaque bitstring en sortie du circuit Haar-random



In [ ]:
nqbits = 15
nlayers = 30
N = 2**nqbits

circuit = haar_random_circuit(nqbits, nlayers)

states, amplitudes = get_result(circuit)

**Question 2** : Si l'état était "complètement aléatoire", quelle distribution de probabilité devraient suivre les bitstrings ? 

REPONDRE ICI:

En réalité, quand un circuit quantique aléatoire est suffisamment profond et qu'il produit un état très intriqué, les amplitudes se comportent comme des nombres complexes aléatoires. Mais du coup, ce n'est pas le cas des probabilités !

Commençons par regarder à quoi ressemble la distribution de probabilité des amplitudes, en plottant un histogramme des parties réelles des amplitudes (les parties imaginaires ont le même comportement)

Vous êtes censé obtenir quelque chose qui ressemble à ça:

<center>

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/distribution_amplitudes.png)


</center>

In [ ]:
plt.hist(N * np.real(amplitudes), bins=50, density=True, alpha=0.6, label="Real part of amplitudes")
plt.ylabel("Density")
plt.legend()
plt.show()

**Question 3** : Quelle distribution de probabilité semblent suivre la partie réelle des amplitudes des bitstrings ?

REPONDRE ICI:

Une conséquence de cette loi aléatoire sur les amplitudes de chaque bitstring est que les probabilités des bitstrings suivent un distribution dite de *Porter-Thomas*:

si l'on considère une bitstring $i$ et $p_i = |\left \langle i | \psi \right \rangle|^2$ la probabilité de mesurer cette bitstring, et $x_i = 2^{\textrm{nqbits}} p_i$ un rescaling de cette probabilité, alors l'histogramme de ces probabilités rescaled suit une distribution exponentielle $P(x) = e^{-x}$



**Question 4** : Compléter le code ci-dessous pour afficher l'histogramme des probabilités (redimmensionnées par un facteur $2^{\textrm{nqbits}}$)

In [ ]:
rescaled_proba = # A compléter pour retourner les probabilités redimensionnées
plt.hist(rescaled_proba, bins=50, density=True, alpha=0.6, label="Probabilities")

x = np.linspace(0, 20, 500)
plt.plot(x, np.exp(-x), c='r',ls='--', label="Porter–Thomas")
plt.legend()
plt.ylabel("Density")
plt.show()

Ainsi, on peut observer que les bitstrings échantillonées à partir d'un circuit Haar-random, ne suivent pas une loi uniforme comme lors d'un sampling classique, mais qu'au contraire, certaines bitstrings sont exponentiellement plus probables que d'autres. Il s'agit d'une conséquence de l'intrication quantique. Pour vérifier cela, nous alons créer un autre type de circuit aléatoire, cette fois sans la moindre intrication:

<center>

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/classical_random.png)

*Circuit aléatoire sans intrications, composés de 5 layers. Chaque layer contient un mur de rotations aléatoires*

</center>

**Question 5** : Implémenter la fonction ```no_intrication_random_circuit``` qui renvoie une QRoutine correspondant à un circuit aléatoire tel que celui donné dans la figure ci-dessus, c'est à dire le circuit aléatoire Haar-random mais sans portes intricatrices. 
Chaque layer doit contenir:
- Une rotation aléatoire à 1 qubit (RX, RY, ou RZ) avec un angle alétoire $\theta \in [0, 2 \pi [$


In [ ]:
def no_intrication_random_circuit(nqbits, depth):
    rout = QRoutine()
    qbits = rout.new_wires(nqbits)

    # A compléter
    
    return rout

Regardons maintenant la distribution de probabilités des bitstrings en sortie de ce circuit

In [ ]:
nqbits = 15
nlayers = 30
N = 2**nqbits

circuit = no_intrication_random_circuit(nqbits, nlayers)

states, amplitudes = get_result(circuit)
rescaled_proba = N * np.abs(amplitudes)**2
plt.hist(rescaled_proba, bins=50, density=True, alpha=0.6, label="Probabilities")

plt.legend()
plt.ylabel("Density")
plt.show()

**Question 6** : A quelle distribution de probabilité corresponds l'histogramme obtenu ?

REPONDRE ICI:

## Exercice 2 : Calcul quantique tolérant aux fautes et gadget de téléportation récursif

Dans le monde du calcul quantique tolérant aux fautes, certaines portes sont en pratique beaucoup plus faciles à réaliser que d'autres. En particulier, les portes suivantes sont considérées comme faciles à réaliser sur un véritable QPU: $H, S=PH(\frac{\pi}{2})$ et $CNOT$.

Cependants, ces seules portes ne sont pas suffisantes pour réaliser tous les circuits quantiques. Pour celà, il faut aussi avoir accès à des rotations de phase arbitraires  $PH(\theta)$

En particulier, on cherche généralement à réaliser la porte $T = PH(\frac{\pi}{4})$, et ce à l'aide des portes mentionnées plus haut et d'un état resource, souvent appellé l'état magique
$\left| \phi \right \rangle = T \left| + \right \rangle = \frac{\left| 0 \right \rangle + e^{i \frac{\pi}{4}} \left| 1 \right \rangle}{\sqrt{2}}$.

Cet état magique est obtenu à l'aide de procédés complexes que nous passerons sous silence ici, et est ensuite injecté dans ce qu'on appelle un gadget de téléportation pour réaliser une porte $T$ sur un état arbitraire, par un procédé appelé téléportation de porte.





<center>

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/gate_teleportation.png)

*Gadget de téléportation. $\left| \theta \right \rangle$ est l'état ressource et $P$ représente la phase-gate $PH$*

</center>

Un gadget de téléportation commence par la préparation d'un état ressource $\left| \theta \right \rangle = PH(\theta) \left| + \right \rangle$

**Question 1**: Compléter le QRoutine ci-dessous pour générer l'état ressource $\left| \theta \right \rangle = PH(\theta) \left| + \right \rangle$. 


In [ ]:
def create_resource_state(theta):
    rout = QRoutine()
    qbit = rout.new_wires(1)

    # A compléter
    
    return rout

**Question 2**: Compléter la fonction ci-dessous pour écrire un programme réalisant ce gadget de téléportation avec l'état magique $\left| \phi \right \rangle$ dans le but d'appliquer une porte T sur le premier qubit. Seules les portes H, CNOT, S et la routine ```create_ressource_state``` doivent être utilisées (Bien entendu les mesures et le contrôle de ces portes pas des bits classiques sont aussi autorisées).

Notez qu'on ajoute une routine d'initialisation sur le qubit 0 pour l'initialiser dans l'état $| \psi \rangle = \cos{\frac{\pi}{6}} | 0 \rangle + \sin{\frac{\pi}{6}} | 1 \rangle$

In [ ]:
# Routine d'initialisation qui crée l'état |psi> = cos(pi/6) |0> + sin(pi/6) |1>
def initialization():
    rout = QRoutine()
    qbit = rout.new_wires(1)
    RY(2*np.pi/6)(qbit[0])
    return rout

def T_gate_gadget():
    prog = Program()
    qbits = prog.qalloc(2)
    bits = prog.calloc(1)

    initialization()(qbits[0])
    
    # A compléter
 
    return prog

In [ ]:
# Vous pouvez vérifiez que votre circuit correspond bien à la figure
circ_T = T_gate_gadget().to_circ()

circ_T.display()

**Question 3**: Quelle est la probabilité de mesurer le qbit 0 dans l'état 0 en sortie de ce circuit ?

REPONDRE ICI:

Nous avons maintenant une manière de réaliser une porte T, à l'aide d'un bit classique et d'un qbit ancillaire préparé dans l'état magique $\left| \theta = \frac{\pi}{4} \right \rangle$. Nous souhaitons maintenant réaliser la porte $\sqrt{T} = PH(\frac{\pi}{8})$ !

Pour ce faire, nous souhaitons utiliser le même gadget de téléportation.

<center>

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/root_t.png)

*Gadget de téléportation pour la porte $\sqrt{T}$*

</center>

**Question 4** : En supposant que nous avons cette fois accès à l'état ressource $\left| \theta = \frac{\pi}{8} \right \rangle$, quelle porte doit-on cette fois appliquer controllée par un bit classique pour réaliser $\sqrt{T}$ ?

REPONDRE ICI:

Il se trouve que cette porte mystère "?" peut se faire à l'aide d'un gadget de téléportation elle même. L'idée est donc de construire un gadget de téléportation récursif, où la porte controllée par un bit classique est elle même un gadget de téléportation, qui utilisera donc son propre qubit ancillaire préparé dans le bon état ressource, et un bit classique supplémentaire de contrôle.

**Question 5** : Compléter la fonction ci-dessous pour écrire un programme réalisant ce gadget de téléportation de téléportation récursif, utilisant deux fois la routine de création d'état resource, et deux bits classiques. Encore une fois, seules les portes H, CNOT, S, et la routine ```create_ressource_state``` doivent être utilisées.

*Indice/Aide*: attention, il n'est pas possible de cc_apply une opération measure, et un cc_apply ne peut être controllé que par un seul bit classique ! Il faut donc bien réfléchir à comment appliquer mesures et contrôle classique dans le gadget de téléportation contrôlé par un bit classique.

Il s'agit d'une question difficile, et toute tentative même infructueuse sera récompensée.

In [ ]:
def recursive_sqrt_T():
    prog = Program()
    qbits = prog.qalloc(3)
    bits = prog.calloc(2)

    # Qubit de donnée: qbits[0]
    # Premier qubit de ressource: qbits[1]
    # Second qubit de ressource: qbits[2]

    initialization()(qbits[0])

    # A compléter

    return prog

In [ ]:
# Vous pouvez vérifier votre circuit ici

circ_sqrt_T = recursive_sqrt_T().to_circ()

circ_sqrt_T.display()